# Web scrapping with python

In [46]:
from bs4 import BeautifulSoup
import requests

### Let's begin with the scrapping

In [47]:
#1. Get the URL
URL_BASE = "https://scrapepark.org/spanish/"
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

#2. HTML Parser
soup = BeautifulSoup(html_obtenido, "html.parser")

In [48]:
type(soup)

bs4.BeautifulSoup

### find() method

In [49]:
primer_h2 = soup.find("h2")
print(primer_h2)

<h2>¿Por qué comprar con nosotros?</h2>


In [50]:
# only text
print(primer_h2.text)

#equivalent to 
print(soup.h2.text)

¿Por qué comprar con nosotros?
¿Por qué comprar con nosotros?


### find_all() method

In [51]:
h2_todos = soup.find_all("h2")
print(h2_todos)

[<h2>¿Por qué comprar con nosotros?</h2>, <h2>
                  #<span>Novedades</span>
</h2>, <h2>Nuestros productos</h2>, <h2>Testimonios de clientes</h2>, <h2 class="heading-container">Precios</h2>]


In [53]:
#If we use the parameter limit=1, we emulate the find() method
h2_uno_solo = soup.find_all("h2", limit=1) 
print(h2_uno_solo)

[<h2>¿Por qué comprar con nosotros?</h2>]


In [52]:
# it is possible to iterate
for seccion in h2_todos:
    print(seccion.text)

¿Por qué comprar con nosotros?

                  #Novedades

Nuestros productos
Testimonios de clientes
Precios


In [54]:
# get_text() for more features
for seccion in h2_todos:
    print(seccion.get_text(strip=True)) 

¿Por qué comprar con nosotros?
#Novedades
Nuestros productos
Testimonios de clientes
Precios


### Using the attributes of the tags

In [55]:
#class
divs = soup.find_all("div",class_ = "heading-container heading-center")

for div in divs:
    print(div)
    print(" ")

<div class="heading-container heading-center" id="about">
<h2>¿Por qué comprar con nosotros?</h2>
</div>
 
<div class="heading-container heading-center" id="products">
<h2>Nuestros productos</h2>
</div>
 
<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>
 
<div class="heading-container heading-center">
<h2>Testimonios de clientes</h2>
</div>
 


In [56]:
# All tags that have the 'src' attribute
src_todos = soup.find_all(src=True)

for elemento in src_todos:
    if elemento["src"].endswith(".jpg"):
        print(elemento)

<img alt="Parque de patinaje" src="../images/slider-bg.jpg"/>
<img alt="Patineta" src="../images/p2.jpg"/>


### Download all the websited images

In [ ]:
url_images = []

for i, imagen in enumerate(src_todos):

    if imagen['src'].endswith("png"):

        print(imagen['src'])
        r = requests.get(f"https://scrapepark.org/spanish/{imagen['src']}")

        with open(f"imagen_{i}.png", "wb") as f:
            f.write(r.content)


### Tables

In [58]:
URL_BASE = 'https://scrapepark.org/spanish/'
URL_TABLA = soup.find_all('iframe')[0]['src']

request_tabla = requests.get(f'{URL_BASE}/{URL_TABLA}')

html_tabla = request_tabla.text
soup_tabla = BeautifulSoup(html_tabla, 'html.parser')
soup_tabla.find('table')

productos_faltantes = soup_tabla.find_all(['th','td'], attrs={'style': 'color: red;'})
productos_faltantes = [talle.text for talle in productos_faltantes]

print(productos_faltantes)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [59]:
divs = soup.find_all('div', class_='detail-box')
productos = []
precios = []

for div in divs:
    if (div.h6 is not None) and ('Patineta' in div.h5.text):
        producto = div.h5.get_text(strip=True)
        precio = div.h6.get_text(strip=True).replace('S','')
        #We can add filters
        print(f'producto: {producto:<16} | precio: {precio}')
        productos.append(producto)
        precios.append(precio)

producto: Patineta nueva1  | precio: $75
producto: Patineta usada2  | precio: $80
producto: Patineta nueva3  | precio: $68
producto: Patineta usada4  | precio: $70
producto: Patineta nueva5  | precio: $75
producto: Patineta nueva6  | precio: $58
producto: Patineta nueva7  | precio: $80
producto: Patineta nueva8  | precio: $35
producto: Patineta nueva9  | precio: $165
producto: Patineta usada10 | precio: $54
producto: Patineta usada11 | precio: $99
producto: Patineta nueva12 | precio: $110


### Changes that depends on the URL

In [61]:
URL_BASE = "https://scrapepark.org/spanish/contact"

for i in range(1,3):
    URL_FINAL = f"{URL_BASE}{i}"
    print(URL_FINAL)
    r = requests.get(URL_FINAL)
    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.h5.text)


https://scrapepark.org/spanish/contact1
Texto que cambia entre páginas en contacto 1 :)
https://scrapepark.org/spanish/contact2
Texto que cambia entre páginas en contacto 2 :)


### Data we don't know where is located in the website

In [62]:
#Regular expressions 
import re

# 1. Get the HTML
URL_BASE = 'https://scrapepark.org/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. HTML Parser
soup = BeautifulSoup(html_obtenido, "html.parser")

telefonos = soup.find_all(string=re.compile("\d+-\d+-\d+"))
telefonos


[': 4-444-4444']

### Moving around the tree

In [63]:
copyrights = soup.find_all(string=re.compile("©"))
copyrights[0]

'© 2022 '

In [64]:
primer_copyright = copyrights[0]
primer_copyright.parent

<p>© 2022 <span>Todos los derechos reservados</span>.
        <a href="https://html.design/" rel="noopener noreferrer" target="_blank">Creado con Free Html Templates</a>.
      </p>

In [65]:
# Another example with elements at the same level
menu = soup.find(string=re.compile("MENÚ"))

# menu.parent
menu.parent.find_next_siblings()


[<ul>
 <li><a href="#">Inicio</a></li>
 <li><a href="#">Acerca</a></li>
 <li><a href="#">Servicios</a></li>
 <li><a href="#">Testimonios</a></li>
 <li><a href="#">Contacto</a></li>
 </ul>]

### Exceptions and webscrapping

In [66]:
strings_a_buscar = ["MENU", "©", "carpinchu", "Patineta"]

for string in strings_a_buscar:
    try:
        resultado = soup.find(string=re.compile(string))
        print(resultado.text)
    except AttributeError:
        print(f"El string '{string}' no fue encontrado")


El string 'MENU' no fue encontrado
© 2022 
El string 'carpinchu' no fue encontrado
Patineta nueva


### Data Storage

In [67]:
productos.insert(0, "productos")
precios.insert(0, "precios")

In [70]:
datos = dict(zip(productos, precios))

In [ ]:
datos.items()

In [71]:
import csv
with open('datos.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(datos.items())